In [6]:
from finiteelementanalysis import visualize as viz
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis import solver_demo_helper_functions as solver_demo
from finiteelementanalysis.solver import hyperelastic_solver
import numpy as np
import matplotlib.pyplot as plt

This notebook will show what happens when we take the small deformation case from the Euler beam solution comparison and apply a large load to it.

We will be using D2_nn6_tri with 100 horizontal elements and 2 vertical elements. Our beam is represented by a 30 x 1 rectangle.

In [7]:
ele_type = "D2_nn6_tri" # change this!
nx = 100 # change this to refine the mesh
ny = 3 # change this to refine the mesh
L = 30
H = 1

coords, connect = pre.generate_rect_mesh_2d(ele_type, 0.0, 0.0, L, H, nx, ny)

Our problem will have a distributed downward load on the top with both left and right ends fixed. Our load and material properties are defined below.

In [8]:
# Identify boundaries
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(coords, connect, ele_type, 0.0, L, 0.0, H)
# 1. Fix left boundary: both u_x and u_y = 0.
fixed_nodes_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0.0, 0.0)
fixed_nodes_right = pre.assign_fixed_nodes_rect(boundary_nodes, "right", 0.0, 0.0)
fixed_nodes = np.hstack((fixed_nodes_left, fixed_nodes_right))
# Assign distributed load on the right boundary
q = 500
dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, -q)
# Assign material properties
E = 1e6
nu = 0.3
# mu = E / (2.0 * (1.0 + nu))
# kappa = E / (3.0 * (1.0 - 2.0 * nu))
mu = E / (2.0 * (1.0 + nu))
#kappa = E / (2.0 * (1.0 - nu))
kappa = E / (3.0 * (1.0 - 2.0 * nu))
material_props = np.array([mu, kappa])
# Assign artificial displacement field
displacement = np.zeros((coords.shape))
for kk in range(0, coords.shape[0]):
    displacement[kk, 0] = coords[kk, 0] * 0.01

# Analytical solution
def solution(x, load_factor):
    w = q*load_factor
    I = H**3/12
    y = 0.5 - w * x**2 / (24*E*I) * (L-x)**2
    return y

# Animated plot
fname = "hw_solver_large_deformation.png"
pre_demo.plot_mesh_2D(fname, ele_type, coords, connect)

We now run the hyperelastic solver over 20 steps and animate the result:

In [9]:
# run the example to look at the results

nr_num_steps = 20
nr_print = True

displacements_all, nr_info_all = hyperelastic_solver(material_props, ele_type, coords.T, connect.T, fixed_nodes, dload_info, nr_print, nr_num_steps, nr_tol=1e-9, nr_maxit=30)

fname = "large-def.gif"
viz.make_deformation_gif(solution, displacements_all, coords, connect, ele_type, fname)

Step 0, load factor = 0.050
Iteration 1, Correction=1.000000e+00, Residual=1.984569e-02, tolerance=1.000000e-09
Iteration 2, Correction=9.683046e-02, Residual=2.545223e+00, tolerance=1.000000e-09
Iteration 3, Correction=7.110130e-02, Residual=2.364541e-02, tolerance=1.000000e-09
Iteration 4, Correction=2.493006e-03, Residual=1.090010e-02, tolerance=1.000000e-09
Iteration 5, Correction=1.016664e-05, Residual=1.304786e-05, tolerance=1.000000e-09
Iteration 6, Correction=7.930774e-11, Residual=2.286522e-10, tolerance=1.000000e-09
Step 1, load factor = 0.100
Iteration 1, Correction=4.351515e-01, Residual=1.984569e-02, tolerance=1.000000e-09
Iteration 2, Correction=7.486384e-02, Residual=1.057301e+00, tolerance=1.000000e-09
Iteration 3, Correction=1.721942e-02, Residual=3.043061e-02, tolerance=1.000000e-09
Iteration 4, Correction=2.814437e-04, Residual=1.674244e-03, tolerance=1.000000e-09
Iteration 5, Correction=1.369727e-07, Residual=4.486116e-07, tolerance=1.000000e-09
Iteration 6, Correct

In [10]:
# Get the final displacements and new positions
final_disp = displacements_all[-1].reshape(-1, 2)
pos = coords + final_disp

# Plot the error
fig = plt.figure()
ax = fig.add_subplot()
ax.scatter(pos[:, 0], final_disp[:, 1] - (solution(pos[:, 0], 1) - 0.5))
ax.set_xlabel('x position')
ax.set_ylabel('Displacement error')
ax.set_title('Difference Between Slender Beam Solution & Numeric Solution')
plt.savefig('large-deformation-diff.png', dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close(fig)